In [1]:
#====================================
# Author: Waleed M. Gondal
# Date: 15, March, 2017
#====================================
import tensorflow as tf
import numpy as np
from config_vgg import VGG16
from utils import write_submission_csv, make_batches, count_images
import os
from IPython import display
import csv
#================================================
# Setting Hyper-Parameters 
#================================================
PRETRAINED_MODEL_PATH = '/home/gmd8si/cats_vs_dogs/run1/weight.ckpt-32'
IMAGE_HEIGHT  = 224
IMAGE_WIDTH   = 224
NUM_CHANNELS  = 3
BATCH_SIZE = 50
N_LABELS = 2
N_EPOCHS = 1  
WEIGHT_PATH = 'vgg16_weights.npz'
TESTSET_PATH = 'test_images_with_id.csv'

# Make Batches from csv files
Make batches of images and their respective name IDs (regarded as labels here to simplify the batch level image naming) 
using standard tensorflow input pipeline

In [2]:
test_image_batch, test_name_batch = make_batches(TESTSET_PATH, N_EPOCHS, IMAGE_HEIGHT,
                                                    IMAGE_WIDTH, BATCH_SIZE, shuffle=False, training = False)

In [3]:
# Count the number of training and test examples
test_count = count_images(TESTSET_PATH)
print ('Test_images: ', test_count)

('Test_images: ', 12501)


In [4]:
# Placeholders for tensorflow graph
images_tf = tf.placeholder( tf.float32, [None, IMAGE_HEIGHT, IMAGE_WIDTH, 3], name="images")
# Configure Network class
detector = VGG16(N_LABELS, WEIGHT_PATH)
output = detector.inference(images_tf, is_training=False, dropout=1.0)
probs = tf.nn.softmax(output)
# Define Network Prediction on output score (Binary Classification)
#prediction = tf.argmax(output, 1)

conv1_1w (?, 224, 224, 64)
conv1_2w (?, 224, 224, 64)
conv2_1w (?, 112, 112, 128)
conv2_2w (?, 112, 112, 128)
conv3_1w (?, 56, 56, 256)
conv3_2w (?, 56, 56, 256)
conv3_3w (?, 56, 56, 256)
conv4_1w (?, 28, 28, 512)
conv4_2w (?, 28, 28, 512)
conv4_3w (?, 28, 28, 512)
conv5_1w (?, 14, 14, 512)
conv5_2w (?, 14, 14, 512)
conv5_3w (?, 14, 14, 512)
fc1 (?, 4096)
fc2 (?, 4096)


# Inference
Network's output for Test dataset 

In [5]:
# Saver Operation to save and restore all variables from checkpoint file
saver = tf.train.Saver() 
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
sess.run(tf.initialize_local_variables())
# Restoring pretrained weights
saver.restore(sess, PRETRAINED_MODEL_PATH)
# Tensorflow Queing process, required to fill queues with data.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

image_ids, predicted_labels = [], [] 
num_iter = int(test_count/BATCH_SIZE)
for i in range (num_iter):
    test_img_batch, test_fnames_batch = sess.run([test_image_batch, test_name_batch])
    model_predictions= sess.run([probs], feed_dict={images_tf: test_img_batch})[0]
    image_ids.append(test_fnames_batch)
    predicted_labels.append(model_predictions)
    display.clear_output(wait=True)
    print 'Batch Count: ', i

Batch Count:  249


# Write CSV Submission file

In [7]:
write_submission_csv('submission_A_dog_prob.csv', image_ids, predicted_labels, num_iter, BATCH_SIZE)

In [8]:
# For Testing the written csv file
path = 'submission_A_dog_prob.csv'
trainfile = open(path, "r")
reader = csv.reader(trainfile)

for row in reader:
    print row

['id', 'label']
['1', '1.0']
['100', '0.0']
['10', '0.0']
['10001', '0.0']
['10004', '0.0005']
['10000', '1.0']
['10002', '0.0']
['1000', '1.0']
['10005', '0.0']
['10003', '1.0']
['10006', '0.0']
['10007', '0.0']
['10008', '0.0']
['10009', '0.0']
['1001', '0.0']
['10010', '1.0']
['10012', '0.0']
['10014', '0.0']
['10013', '0.0']
['10016', '0.0']
['10011', '1.0']
['10015', '0.9997']
['10017', '0.0']
['10020', '0.0']
['10019', '0.0']
['10018', '0.0']
['10023', '1.0']
['1002', '1.0']
['10024', '1.0']
['10022', '1.0']
['10025', '0.9999']
['10028', '0.0']
['10029', '1.0']
['10027', '1.0']
['10026', '1.0']
['10021', '0.0']
['1003', '1.0']
['10031', '0.0']
['10033', '0.0']
['10030', '1.0']
['10034', '1.0']
['10032', '0.0']
['10035', '0.9956']
['10037', '0.0']
['10038', '1.0']
['10036', '1.0']
['10040', '0.0']
['1004', '1.0']
['10039', '1.0']
['10041', '1.0']
['10042', '0.9849']
['10045', '0.2336']
['10043', '1.0']
['10044', '0.0']
['10047', '1.0']
['10046', '1.0']
['10048', '1.0']
['1005', '0